# Orders

In [ ]:
# Auto reload imported module everytime a jupyter cell is executed (handy for olist.order.py updates)
%load_ext autoreload
%autoreload 2

In [ ]:
# Import usual modules
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Import olist data
from olist.data import Olist
olist=Olist()
data=olist.get_data()
matching_table = olist.get_matching_table()

In [3]:
'testing 3'


'testing 3'